In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_fotball/tensors/"
tensor_dir = "./test_tensors/layer4/"
output_dir = "./measurements/"
layer =4

test_tensors = os.listdir(tensor_dir)

In [3]:
def sort_fun(tensor_name):
    return int(tensor_name.split("_")[2].split(".")[0])

test_tensors.sort(key=sort_fun)

In [4]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [5]:
for tensor_name in test_tensors[10:20]:
    print("************* Testing tensor: "+tensor_name+"***************")
    split_layer.append(layer)
    tensor = torch.load(tensor_dir+tensor_name)
    # tensor = tensor * (abs(tensor)>0.001)
    tensor = tensor[0]

    # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(get_tensor_pictoriality(tensor))
    tensor_regularity.append(get_tensor_regularity(tensor))
    tensor_decomposability.append(get_tensor_decomposability(tensor))

    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    jpeg_ratio.append(ratio_jpeg)
    jpeg_snr.append(snr_jpeg)
    jpeg_mse.append(mse_jpeg)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    decomposition_ratio.append(ratio_decomposition)
    decomposition_snr.append(snr_decomposition)
    decomposition_mse.append(mse_decomposition)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    regression_ratio.append(ratio_regression)
    regression_snr.append(snr_regression)
    regression_mse.append(mse_regression)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    sketchml_ratio.append(ratio_sketchml)
    sketchml_snr.append(snr_sketchml)
    sketchml_mse.append(mse_sketchml)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


************* Testing tensor: tensor_l4_10.pt***************
Rank: 103.406250, Sparsity: 0.000000, Picoriality: 0.732649, Regularity: 0.683971, Decomposability: -0.988582
Doing JPEG compression
Ratio: 5.994709, SNR: 31.308930, MSE: 0.000001
Doing Decomposition compression
Ratio: 0.502871, SNR: 106.969967, MSE: 0.000000
Doing Regression compression
Ratio: 15.929308, SNR: 9.181169, MSE: 0.000149
Doing SketchML compression
Ratio: 4.385884, SNR: 1.462460, MSE: 0.000882
************* Testing tensor: tensor_l4_11.pt***************
Rank: 103.531250, Sparsity: 0.000000, Picoriality: 0.743711, Regularity: 0.696818, Decomposability: -0.990986
Doing JPEG compression
Ratio: 5.715876, SNR: 30.277205, MSE: 0.000001
Doing Decomposition compression
Ratio: 0.502264, SNR: 103.094006, MSE: 0.000000
Doing Regression compression
Ratio: 15.929308, SNR: 9.381635, MSE: 0.000152
Doing SketchML compression
Ratio: 4.392744, SNR: 1.502746, MSE: 0.000932
************* Testing tensor: tensor_l4_12.pt***************

In [6]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'decomposability':tensor_decomposability,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [7]:
df.to_csv(output_dir+"layer"+str(layer)+".csv",index=False, header=True)

In [8]:
df.head()

,split_layer,rank,sparsity,regularity,pictoriality,decomposability,jpeg_ratio,jpeg_snr,jpeg_mse,decomposition_ratio,decomposition_snr,decomposition_mse,regression_ratio,regression_snr,regression_mse,sketchml_ratio,sketchml_snr,sketchml_mse
0,4,103.40625,0.0,0.683971,0.732649,-0.988582,5.994709,31.308930,9.139889e-07,0.502871,106.969967,2.482201e-14,15.929308,9.181169,0.000149,4.385884,1.462460,0.000882
1,4,103.53125,0.0,0.696818,0.743711,-0.990986,5.715876,30.277205,1.235585e-06,0.502264,103.094006,6.459405e-14,15.929308,9.381635,0.000152,4.392744,1.502746,0.000932
2,4,103.56250,0.0,0.687721,0.732474,-0.991587,5.840000,32.068179,8.027421e-07,0.502112,104.013119,5.129580e-14,15.929308,9.453684,0.000147,4.386078,1.509205,0.000913
3,4,103.53125,0.0,0.696026,0.737915,-0.990986,6.472166,30.534499,9.669873e-07,0.502264,104.128408,4.226976e-14,15.929308,10.237923,0.000104,4.390682,1.419251,0.000789
4,4,103.62500,0.0,0.685419,0.729381,-0.992788,5.582767,31.207726,1.013842e-06,0.501809,106.922674,2.719417e-14,15.929308,9.134751,0.000163,4.391978,1.427578,0.000964
